# Przykład użycia narzędzia AutoGen


## Importuj Potrzebne Pakiety


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Tworzenie klienta

W tym przykładzie użyjemy [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) do uzyskania dostępu do LLM.

`model` jest zdefiniowany jako `gpt-4o-mini`. Spróbuj zmienić model na inny dostępny w marketplace GitHub Models, aby zobaczyć różne wyniki.

Jako szybki test uruchomimy prostą komendę - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiowanie funkcji

W tym przykładzie damy agentowi dostęp do narzędzia, które jest funkcją zawierającą listę dostępnych miejsc na wakacje oraz ich dostępność.

Możesz wyobrazić sobie, że jest to scenariusz, w którym agent podróży ma dostęp do bazy danych dotyczącej podróży, na przykład.

Przechodząc przez ten przykład, śmiało próbuj definiować nowe funkcje i narzędzia, które agent może wywoływać.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definiowanie narzędzia funkcji  
Aby agent mógł używać `vacation_destinations` jako `FunctionTool`, musimy zdefiniować je jako takie.  

Podamy również opis narzędzia, który pomoże agentowi zidentyfikować, do czego to narzędzie służy w kontekście zadania zleconego przez użytkownika.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definiowanie agenta

Teraz możemy stworzyć agenta w poniższym kodzie. Definiujemy `system_message`, aby przekazać agentowi instrukcje dotyczące pomocy użytkownikom w znajdowaniu miejsc na wakacje.

Ustawiamy również parametr `reflect_on_tool_use` na wartość true. Dzięki temu LLM może wykorzystać odpowiedź narzędzia i przesłać ją w naturalnym języku.

Możesz ustawić ten parametr na wartość false, aby zobaczyć różnicę.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Uruchamianie agenta

Teraz możemy uruchomić agenta z początkową wiadomością użytkownika, w której prosi o podróż do Tokio.

Możesz zmienić tę destynację, aby zobaczyć, jak agent reaguje na dostępność miasta.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
